In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


In [2]:
# Ignore warnings
# import warnings
# warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [3]:
#load data
data_dir = 'dogs-cats-mini/dogs-cats-mini'
# split images into train and test sets with 80/20 ratio, they are not in separate folders
# so we need to create a list of images and their labels
# we will use 80% of images for training and 20% for testing

# create a list of images and their labels
images = []
labels = []
for file in os.listdir(data_dir)[10000:15000]:
    if file.startswith('cat'):
        images.append(os.path.join(data_dir, file))
        labels.append(0)
    elif file.startswith('dog'):
        images.append(os.path.join(data_dir, file))
        labels.append(1)

# create a dataframe with images and labels
df = pd.DataFrame({'image': images, 'label': labels})

# shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

# split the dataframe into train and test sets
train_size = int(0.8 * len(df))
test_size = len(df) - train_size
train_df, test_df = df.iloc[:train_size], df.iloc[train_size:]

# create a list of images and labels for train and test sets
train_images = train_df['image'].tolist()
train_labels = train_df['label'].tolist()
test_images = test_df['image'].tolist()
test_labels = test_df['label'].tolist()

In [4]:
# create a dataset class
class CatsDogsDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [5]:
# create a transform
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# create train and test datasets
train_dataset = CatsDogsDataset(train_images, train_labels, transform)
test_dataset = CatsDogsDataset(test_images, test_labels, transform)

# create train and test dataloaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, num_workers=4)


In [6]:
# create a function to show images
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
# dataiter = iter(train_loader)
# images, labels = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(' '.join('%5s' % labels[j] for j in range(4)))


In [7]:
# create a neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 29 * 29, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 29 * 29)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
net = Net()
net.to(device)

# define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
# train the network
for epoch in range(2):  # loop over the dataset multiple times
    net.train()
    print('Epoch {}/{}'.format(epoch, 2 - 1))
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        print('Batch {}/{}'.format(i, len(train_loader) - 1))
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0
        
    # test the network
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
    print('Accuracy of the network on the 100 test images: %d %%' % (
        100 * correct / total))
    
print('Finished Training')

Epoch 0/1


In [ ]:
# save the model
PATH = './cats_dogs_net.pth'
torch.save(net.state_dict(), PATH)

# load the model
net = Net()
net.load_state_dict(torch.load(PATH))

# test the network on test data
dataiter = iter(test_loader)
images, labels = dataiter.next()